# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.124967e+02     1.412722e+00
 * time: 0.7300479412078857
     1     1.058049e+01     1.009643e+00
 * time: 2.110646963119507
     2    -1.207662e+01     1.062056e+00
 * time: 2.786134958267212
     3    -3.417681e+01     7.335594e-01
 * time: 3.765587091445923
     4    -4.783648e+01     5.189986e-01
 * time: 4.755934000015259
     5    -5.702807e+01     1.890584e-01
 * time: 5.715878963470459
     6    -5.983332e+01     1.360607e-01
 * time: 6.36284613609314
     7    -6.091038e+01     4.883278e-02
 * time: 7.018033981323242
     8    -6.133555e+01     6.393858e-02
 * time: 7.680139064788818
     9    -6.162136e+01     3.993785e-02
 * time: 8.348122119903564
    10    -6.184487e+01     2.810741e-02
 * time: 9.001212120056152
    11    -6.201996e+01     2.151305e-02
 * time: 9.642990112304688
    12    -6.208909e+01     1.506683e-02
 * time: 10.284526109695435
    13    -6.215346e+01     1.386653e-02
 * time: 10.957452058792114
    1

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671068
    AtomicLocal         -18.8557674
    AtomicNonlocal      14.8522643
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485370 
    Xc                  -19.3336819

    total               -62.261666462120
